In [1]:
import pandas as pd

# Load the uploaded Excel file
file_path = "/Users/maitreyieadep/Documents/Github_Projects/E_Commerce_project/amazon_sales_data 2025 .xlsx"
excel_file = pd.ExcelFile(file_path)

# Display all sheet names
sheet_names = excel_file.sheet_names

# Preview first few rows of each sheet
sheet_previews = {sheet: excel_file.parse(sheet).head() for sheet in sheet_names}
sheet_names, sheet_previews


(['amazon_sales_data 2025 2'],
 {'amazon_sales_data 2025 2':   Order ID       Date        Product     Category  Price  Quantity  \
  0  ORD0001 2025-03-14  Running Shoes     Footwear     60         3   
  1  ORD0002 2025-03-20     Headphones  Electronics    100         4   
  2  ORD0003 2025-02-15  Running Shoes     Footwear     60         2   
  3  ORD0004 2025-02-19  Running Shoes     Footwear     60         3   
  4  ORD0005 2025-03-10     Smartwatch  Electronics    150         3   
  
     Total Sales  Customer Name Customer Location Payment Method     Status  
  0          180     Emma Clark          New York     Debit Card  Cancelled  
  1          400  Emily Johnson     San Francisco     Debit Card    Pending  
  2          120       John Doe            Denver     Amazon Pay  Cancelled  
  3          180  Olivia Wilson            Dallas    Credit Card    Pending  
  4          450     Emma Clark          New York     Debit Card    Pending  })

In [16]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# ---------------------------------------------
# Load the main dataset again
# ---------------------------------------------
df = excel_file.parse("amazon_sales_data 2025 2")

# ---------------------------------------------
# Clean and Format Date
# ---------------------------------------------
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Add formatted date for clean export/display
df['Date_formatted'] = df['Date'].dt.strftime('%Y-%m-%d')

# Add Year, Month, Day for Tableau filtering and seasonal analysis
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

# Drop rows with missing critical values
df = df.dropna(subset=['Date', 'Price', 'Quantity', 'Total Sales'])

# ---------------------------------------------
# Add Calculated Fields
# ---------------------------------------------
df['Revenue'] = df['Total Sales']
df['Profit'] = df['Revenue'] * 0.3  # Assuming 30% margin
df['YearMonth'] = df['Date'].dt.to_period('M').astype(str)

# ---------------------------------------------
# RFM Segmentation
# ---------------------------------------------
now = datetime(2025, 6, 26)
rfm = df.groupby('Customer Name').agg({
    'Date': lambda x: (now - x.max()).days,
    'Order ID': 'nunique',
    'Revenue': 'sum'
}).reset_index()
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Normalize and segment using KMeans
X = StandardScaler().fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])
kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Segment'] = kmeans.fit_predict(X)

# ---------------------------------------------
# 7-Day Moving Average Forecast
# ---------------------------------------------
revenue_daily = df.groupby('Date')['Revenue'].sum().reset_index()
revenue_daily['7_day_avg'] = revenue_daily['Revenue'].rolling(window=7, min_periods=1).mean()
revenue_daily['Date_formatted'] = revenue_daily['Date'].dt.strftime('%Y-%m-%d')

# ---------------------------------------------
# Replace Date with Date_formatted for clean export
# ---------------------------------------------
df = df.drop(columns=['Date'])  # remove datetime column with time
df = df.rename(columns={'Date_formatted': 'Date'})  # replace with clean YYYY-MM-DD string

revenue_daily = revenue_daily.drop(columns=['Date'])
revenue_daily = revenue_daily.rename(columns={'Date_formatted': 'Date'})

# ---------------------------------------------
# Export All Results to Excel
# ---------------------------------------------
output_path = "/Users/maitreyieadep/Documents/Github_Projects/E_Commerce_project/Amazon_Dashboard_Processed_NoProphet.xlsx"

with pd.ExcelWriter(output_path) as writer:
    df.to_excel(writer, sheet_name='Cleaned_Sales_Data', index=False)
    rfm.to_excel(writer, sheet_name='RFM_Segmentation', index=False)
    revenue_daily.to_excel(writer, sheet_name='Revenue_7Day_Avg', index=False)

print("Data export completed successfully.")
print(f"File saved to: {output_path}")


✅ Data export completed successfully.
📂 File saved to: /Users/maitreyieadep/Documents/Github_Projects/E_Commerce_project/Amazon_Dashboard_Processed_NoProphet.xlsx


In [22]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ---------------------------------------------
# Load cleaned sales data
# ---------------------------------------------
file_path = "/Users/maitreyieadep/Documents/Github_Projects/E_Commerce_project/Amazon_Dashboard_Processed_NoProphet.xlsx"
df = pd.read_excel(file_path, sheet_name='Cleaned_Sales_Data')

# Convert 'Date' column to datetime (safe since now it is in YYYY-MM-DD)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# ---------------------------------------------
# Anomaly Detection in Daily Revenue
# ---------------------------------------------
daily_rev = df.groupby('Date')['Revenue'].sum().reset_index()

iso_model = IsolationForest(contamination=0.05, random_state=42)
daily_rev['anomaly'] = iso_model.fit_predict(daily_rev[['Revenue']])
daily_rev['Anomaly Flag'] = daily_rev['anomaly'].map({1: 'Normal', -1: 'Anomaly'})

# Format date for clean export
daily_rev['Date'] = daily_rev['Date'].dt.strftime('%Y-%m-%d')

# ---------------------------------------------
# Customer Churn Prediction
# ---------------------------------------------
# Get last purchase and days since last purchase
df['LastPurchase'] = df.groupby('Customer Name')['Date'].transform('max')
df['DaysSinceLast'] = (datetime(2025, 6, 26) - df['LastPurchase']).dt.days

# Define churn: customers with no purchase in last 60 days
df['Churned'] = df['DaysSinceLast'].apply(lambda x: 1 if x > 60 else 0)

# Create RFM features
rfm = df.groupby('Customer Name').agg({
    'Date': lambda x: (datetime(2025, 6, 26) - x.max()).days,
    'Order ID': 'nunique',
    'Revenue': 'sum'
}).reset_index()
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

# Add churn labels
churn_data = df[['Customer Name', 'Churned']].drop_duplicates().rename(columns={'Customer Name': 'CustomerID'})
rfm = rfm.merge(churn_data, on='CustomerID', how='left')

# Prepare X and y
X = rfm[['Recency', 'Frequency', 'Monetary']]
y = rfm['Churned']

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle case when only one churn class exists
if y.nunique() > 1:
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_train, y_train)
    rfm['Churn_Probability'] = clf.predict_proba(X_scaled)[:, 1]
else:
    default_prob = float(y.unique()[0])
    rfm['Churn_Probability'] = default_prob

# ---------------------------------------------
# Export AI Insights to Excel
# ---------------------------------------------
output_path = "/Users/maitreyieadep/Documents/Github_Projects/E_Commerce_project/Amazon_AI_Insights.xlsx"

with pd.ExcelWriter(output_path) as writer:
    daily_rev.to_excel(writer, sheet_name='Revenue_Anomalies', index=False)
    rfm.to_excel(writer, sheet_name='Customer_Churn', index=False)

print("AI insights exported successfully.")
print(f"File saved to: {output_path}")



✅ AI insights exported successfully.
📂 File saved to: /Users/maitreyieadep/Documents/Github_Projects/E_Commerce_project/Amazon_AI_Insights.xlsx
